# INITIALIZATION

In [9]:
# Loading some modules
%load_ext autoreload
%autoreload 2

# Getting basic libraries:
import os, sys, math
import numpy as np
from Bio.PDB import *

# Libraries for structure checking: --> 
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]
args = cts.set_defaults(base_dir_path,{'notebook':True})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# PREPARATION

All data should be in a folder named assignment_data

    1 - Obtain the required structure from the PDB

In [10]:
# Loading the protein into a variable
f=open("./assignment_data/6m0j.pdb", "r")
parser = PDBParser()
structure = parser.get_structure("6m0j",f)
st=structure

    2 - Check at PDB which is the composition of a “Biological unit”. Remove all chains but those involved in the biological unit, if necessary

In [11]:
# Preparing the paths
base_path = './assignment_data/'
args['output_format'] = "pdb"
args['keep_canonical'] = False
args['input_structure_path'] = base_path + '6m0j.cif'
args['output_structure_path'] = base_path + '6m0j_fixed.pdb'
args['output_structure_path_charges'] = base_path + '6m0j_fixed.pdbqt'
args['time_limit'] = False
args['nocache'] = False
args['copy_input'] = False
args['build_warnings'] = False
args['debug'] = False
args['verbose'] = False
args['coords_only'] = False
args['overwrite'] = False

In [12]:
# Intializing the checking engine, loading the structure and showing info
st_c = StructureChecking(base_dir_path, args)

Canonical sequence for model 0:
Structure ./assignment_data/6m0j.cif loaded
 PDB id: 6M0J 
 Title: Crystal structure of 2019-nCoV spike receptor-binding domain bound with ACE2
 Experimental method: X-RAY DIFFRACTION
 Keywords: VIRAL PROTEIN/HYDROLASE
 Resolution (A): 2.4500

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  876
 Num. residues with ins. codes:  0
 Num. residues with H atoms: 0
 Num. HETATM residues:  85
 Num. ligands or modified residues:  5
 Num. water mol.:  80
 Num. atoms:  6543
Metal/Ion residues found
 ZN A901
Small mol ligands found
NAG A902
NAG A903
NAG A904
NAG E601


    3 - Remove all heteroatoms


In [13]:
#Remove Hydrogen
st_c.rem_hydrogen()
#Remove water
st_c.water("yes")
#Remove metals
st_c.metals("All")
#Remove ligands
st_c.ligands("All")

Running rem_hydrogen.
No residues with Hydrogen atoms found
Running water. Options: yes
Detected 80 Water molecules
Canonical sequence for model 0:
Removed 80 Water molecules
Running metals. Options: All
Found 1 Metal ions
  ZN A901.ZN 
Canonical sequence for model 0:
Metal Atoms removed All (1)
Running ligands. Options: All
Detected 4 Ligands
 NAG A902
 NAG A903
 NAG A904
 NAG E601
Canonical sequence for model 0:
Ligands removed All (4)


    4 - Perform a quality checking on the structures, and add missing side-chains and hydrogen atoms and atom charges, using the biobb_structure_checking module

In [14]:
#Fix amides
st_c.amide("All")
#fix the chirality of some aa
st_c.chiral("All") 
#fix the backbone
st_c.backbone('--fix_atoms All --fix_chain none --add_caps none')
#detects and rebuilds missing protein side chains
st_c.fixside("All")
#add hydrogens
st_c.add_hydrogen("auto")

#you could also do everything with st_c.checkall() but we did it manually so its clearer what we do
st_c._save_structure(args['output_structure_path'])

f.close()

Running amide. Options: All
Found 6 unusual contact(s) involving amide atoms
 VAL A59.O    ASN A63.OD1     2.784 A
 ALA A80.O    GLN A101.OE1    2.931 A
 GLN A81.OE1  ASN A103.OD1    2.859 A
 ASN A134.ND2 ASN A137.N      2.987 A
 GLU A150.O   ASN A154.OD1    2.871 A
 ARG E357.NH1 ASN E394.ND2    2.964 A
Amide residues fixed All (7)
Rechecking
Found 4 unusual contact(s) involving amide atoms
 GLN A81.NE2  ASN A103.ND2    2.859 A
 ASN A103.OD1 ASN A194.OD1    2.485 A
 ARG E357.NH1 ASN E394.ND2    3.058 A
 ASN E394.OD1 GLU E516.OE2    2.870 A
Running chiral. Options: All
Found no residues with incorrect side-chain chirality
Running backbone. Options: --fix_atoms All --fix_chain none --add_caps none
Found 2 Residues with missing backbone atoms
 ASP A615   OXT
 GLY E526   OXT
No backbone breaks
No unexpected backbone links
Capping terminal ends
True terminal residues: A19,A615,E333,E526
No caps added
Fixing missing backbone atoms
Adding missing backbone atoms
ASP A615
  Adding new atom OXT


## STEP 1

In [15]:
#Finding out atoms in both chains
def chain_atoms(structure, dt):
    model =structure[0]
    t=0
    chain1_atoms=[]
    chain2_atoms=[]
    for chain in model:
        if t==0:
            t=1
            for residual in chain.get_residues():
                for atom1 in residual:
                    chain1_atoms.append(atom1)
        else:
            for residual in chain.get_residues():
                for atom2 in residual:
                    chain2_atoms.append(atom2)
    return chain_comparison(chain1_atoms, chain2_atoms, dt)

#comparing atoms of both chains
def chain_comparison(chain1_atoms, chain2_atoms, dt):
    interface_residues=set()
    for a1 in chain1_atoms:
        for a2 in chain2_atoms:
            if a1.get_parent().id==a1.get_parent().id and a2.get_parent().id == a2.get_parent().id:
                distance=a1-a2
            if distance<=dt:#dt is the treshold of the distance
                interface_residues.add(a1.get_parent().id[1])
                interface_residues.add(a2.get_parent().id[1])
    return interface_residues

dt=5
print(chain_atoms(structure, dt))

KeyboardInterrupt: 

## Step 2

Step 2 preparation

In [ ]:
from modules_classes import *

def calc_solvation(st, res):
    '''Solvation energy based on ASA'''
    solv = 0.
    for at in res.get_atoms():
        if 'EXP_NACCESS' not in at.xtra:
            continue
        s = float(at.xtra['EXP_NACCESS'])* at.xtra['vdw'].fsrf
        solv += s
    return solv


def vdw_int(at1, at2, r):
    '''Vdw interaction energy between two atoms'''
    eps12 = math.sqrt(at1.xtra['vdw'].eps * at2.xtra['vdw'].eps)
    sig12_2 = at1.xtra['vdw'].sig * at2.xtra['vdw'].sig
    return 4 * eps12 * (sig12_2**6/r**12 - sig12_2**3/r**6)

def MH_diel(r):
    '''Mehler-Solmajer dielectric'''
    return 86.9525 / (1 - 7.7839 * math.exp(-0.3153 * r)) - 8.5525

def elec_int(at1, at2, r):
    '''Electrostatic interaction energy between two atoms at r distance'''
    return 332.16 * at1.xtra['charge'] * at2.xtra['charge'] / MH_diel(r) / r

def calc_int_energies(st, res):
    elec = 0.
    vdw = 0.
    for at1 in res.get_atoms():
        for at2 in st.get_atoms():
        # skip same chain atom pairs
            if at2.get_parent().get_parent() != res.get_parent():
                r = at1 - at2
                e = elec_int(at1, at2, r)
                elec += e
                e = vdw_int(at1, at2, r)
                vdw += e
    return elec, vdw

def add_atom_parameters(st, res_lib, ff_params):
    ''' Adds parameters from libraries to atom .xtra field
        For not recognized atoms, issues a warning and put default parameters
    '''
    for at in st.get_atoms():
        resname = at.get_parent().get_resname()
        params = res_lib.get_params(resname, at.id)
        if not params:
            #print("WARNING: residue/atom pair not in library ("+atom_id(at) + ')')
            at.xtra['atom_type'] = at.element
            at.xtra['charge'] = 0
        else:
            at.xtra['atom_type'] = params.at_type
            at.xtra['charge'] = params.charge
        at.xtra['vdw'] = ff_params.at_types[at.xtra['atom_type']]

Step 2 execution

In [ ]:

wd=str(input("Input working directory: "))
residue_library = ResiduesDataLib(wd+'/assignment_data/parameters_step2.lib')
ff_params = VdwParamset(wd+'/assignment_data/parameters_vanderw.txt')
NACCESS_BINARY =wd+'/soft/NACCESS/naccess'

add_atom_parameters(st, residue_library,ff_params)
def chain_atoms2(structure, dt):
    model =structure[0]
    t=0
    chain1_atoms=set()
    chain2_atoms=set()
    for chain in model:
        if t==0:
            t=1
            chain1=chain
        else: chain2=chain
    NeighborSearch_chain2 = NeighborSearch(list(chain2.get_atoms()))
    for residual1 in chain1.get_residues():
        for atom1 in residual1:
            Neighbor_atom_chain2 = NeighborSearch_chain2.search(atom1.coord, dt)
            for atom2 in Neighbor_atom_chain2:
                residual2=atom2.get_parent()
                chain1_atoms.add(residual1)
                chain2_atoms.add(residual2) 
    return calc(chain1_atoms, chain2_atoms)

def calc(chain1_atoms, chain2_atoms):
    s=sA=sE=e=v=0
    for res in chain1_atoms:
        s += calc_solvation(st[0], res)
        sA += calc_solvation(st[0]['A'], res)
        E, V = calc_int_energies(st, res)
        e+=E
        v+=V
    for res in chain2_atoms:
        s += calc_solvation(st[0], res)
        sE += calc_solvation(st[0]['E'], res)
        E, V = calc_int_energies(st, res)
        e+=E
        v+=V
    G = e + v + s - sA - sE
    print("Electrostatic interactions:", e)
    print("Van der Waals interactions", v)
    print("Solvation A-B complex:", s)
    print("Solvation of A:", sA)
    print("Solvation of B", sE)
    return("Interaction energy between components in A-B complex:",G)

print(chain_atoms2(structure, dt))

## Step 3

Preparing our libraries

In [16]:
# Loading Libraries
# loading residue library from data/aaLib.lib
from modules_classes import ResiduesDataLib
from modules_classes import VdwParamset
import step2m2_energies as en
from Bio.PDB.NACCESS import NACCESS_atomic
import os

dir = os.getcwd() # Getting current directory

residue_library = ResiduesDataLib(dir+'/assignment_data/parameters_step2.lib')

# loading VdW parameters
ff_params = VdwParamset(dir+'/assignment_data/parameters_vanderw.txt')

Defining some important variables

In [17]:
pdb_file = dir+'/assignment_data/6m0j_fixed.pdb'
cutoff_dist = 5.0 # Setting the distance, in our case we used a distance of 5.0 amstrong
NACCESS_BINARY = dir+'/soft/NACCESS/naccess'

Loading the structure from the pdb file and preparing our data

In [18]:
parser = PDBParser(PERMISSIVE=1)
print('Parsing', pdb_file)

# load structure from PDB file of PDB ifle handler
st = parser.get_structure('STR', pdb_file)

# assign data types, and charges from libraries
# We will use the xtra attribute in Bio.PDB.Atom to hold the new data
# Possible errors on N-term and C-Term atoms
# Possible errors on HIS alternative forms

en.add_atom_parameters(st, residue_library, ff_params)

Parsing /home/jaume/Desktop/Year_2/Biophysics/Project:Energy_Analysis/Biophysics_A1/assignment_data/6m0j_fixed.pdb


SURFACES

In [19]:
# Calculating surfaces
# The specific PATH to naccess script (in soft) is needed
# ASA goes to .xtra field directly

srf = NACCESS_atomic(st[0], naccess_binary=NACCESS_BINARY)

# Prepare surfaces for the separate chains
# Alternatively the twp PDB files can be prepared outside and parsed here

io = PDBIO()
st_chains = {}
# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
	def __init__(self, chid):
		self.id = chid

	def accept_chain(self, chain):
		if chain.id == self.id:
			return 1
		else:
			return 0

for ch in st[0]:
	io.set_structure(st)
	io.save('tmp.pdb', SelectChain(ch.id))
	st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
	en.add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
	srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
os.remove('tmp.pdb')

INTERFACE RESIDUES

In [20]:
## Interface residues
if cutoff_dist > 0.: # Applying the cutoff
	interface = en.get_interface(st, cutoff_dist)

## Initiatlize Energy aggregates
elec = {}
elec_ala = {}

vdw = {}
vdw_ala = {}

solvAB = {}
solvAB_ala = {}

solvA = {}
solvA_ala = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

## We get the chsin ids,not always they are A and B
chids = []
for ch in st[0]:
	chids.append(ch.id)
	totalSolvMon[ch.id] = 0

total = 0.

Creation of a file and getting the output for the different energy types and their associated residue for the INTERACTION ENERGY BASED ON INTERFACE RESIDUES

In [21]:
print('------------------------------------------------------------------------------------------------------------------')
print(f'\nInteraction energy based in interface residues ONLY')

# Opening/creating the file in writting mode
with open("inter_en_res.csv", "w") as file:

	# Header with format (as output in the terminal & write in tha file):
	print(
		'D#{:11}  {:11s} {:11s} {:11s} {:11s} | {:11s} {:11s} {:11s} {:11s}'.format(
			'res_id',
			'elec_res', 'vdw_res', 'solv_AB_res', 'solv_A_res',
			'elec_ala', 'vdw_ala', 'solv_AB_ala', 'solv_A_ala'))

	file.write(
		'D#{:11},{:11s},{:11s},{:11s},{:11s}, - ,{:11s},{:11s},{:11s},{:11s}\n'.format(
			'res_id',
			'elec_res', 'vdw_res', 'solv_AB_res', 'solv_A_res',
			'elec_ala', 'vdw_ala', 'solv_AB_ala', 'solv_A_ala'))

	# Iteration through the chain
	for ch in st[0]:

		# Getting residues and iterating through them
		for res in ch.get_residues():

			# Applying the cut distance filter so that the distance is consistent (not 0)
			if cutoff_dist > 0 and res not in interface[ch.id]:
				continue

			# Calculation of energies and assigning them to a variable using the following functions
			elec[res], elec_ala[res], vdw[res], vdw_ala[res] = en.calc_int_energies(st[0], res)
			solvAB[res], solvAB_ala[res] = en.calc_solvation(st[0], res)
			solvA[res], solvA_ala[res] = en.calc_solvation(
				st_chains[ch.id],
				st_chains[ch.id][0][ch.id][res.id[1]]
			)

			# Calculation of total energies
			totalIntElec += elec[res]
			totalIntVdw += vdw[res]
			totalSolv += solvAB[res]
			totalSolvMon[ch.id] += solvA[res]
			total += elec[res] + vdw[res] + solvAB[res] - solvA[res]

			# Main body of all the info for each residue with it's csv format (as output in the terminal & write in tha file)
			print(
				'D#{:11}  {:11.4f} {:11.4f} {:11.4f} {:11.4f} | {:11.4f} {:11.4f} {:11.4f} {:11.4f}'.format(
					en.residue_id(res),
					elec[res], vdw[res], solvAB[res], solvA[res],
					elec_ala[res], vdw_ala[res], solvAB_ala[res], solvA_ala[res]))
			
			file.write(
				'D#{:11},{:11.4f},{:11.4f},{:11.4f},{:11.4f}, | ,{:11.4f},{:11.4f},{:11.4f},{:11.4f}\n'.format(
					en.residue_id(res),
					elec[res], vdw[res], solvAB[res], solvA[res],
					elec_ala[res], vdw_ala[res], solvAB_ala[res], solvA_ala[res]))

print('\n------------------------------------------------------------------------------------------------------------------')


------------------------------------------------------------------------------------------------------------------

Interaction energy based in interface residues ONLY
D#res_id       elec_res    vdw_res     solv_AB_res solv_A_res  | elec_ala    vdw_ala     solv_AB_ala solv_A_ala 
D#SER A19           3.6318     -0.9238     -3.4036     -4.4436 |      2.4167     -0.8400     -2.1335     -3.1735
D#GLN A24           0.3767     -1.7267     -1.8006     -2.4184 |     -0.1379     -2.2663      0.0075      0.1122
D#THR A27          -0.0266     -6.4291     -0.7248      0.3872 |      1.2623     -3.2416      0.0444      0.1200
D#PHE A28          -0.0999     -3.3071      0.5986      1.6252 |     -0.0112     -1.9429     -0.0005      0.0182
D#ASP A30          -2.7963     -1.5358     -1.8000     -3.3357 |     -0.8539      0.9669     -0.0448      0.0908
D#LYS A31           4.9173     -8.2069     -2.2030     -1.5507 |      0.1407     -3.1928      0.0038      0.2487
D#HIE A34          -0.0567     -2.5958   

Printing a summary:

In [22]:
print('-----------------------------------------------------------------------')
print(f'\nTOTAL ENERGIES of interaction energy based in interface residues ONLY')
print('{:20}: {:11.4f}'.format('Total Elec Int.', totalIntElec))
print('{:20}: {:11.4f}'.format('Total Vdw Int.', totalIntVdw))
print('{:20}: {:11.4f}'.format('Total Solv AB', totalSolv))
print('{:19}{}: {:11.4f}'.format('Total Solv ', chids[0], totalSolvMon[chids[0]]))
print('{:19}{}: {:11.4f}'.format('Total Solv ', chids[1], totalSolvMon[chids[1]]))
print('{:20}: {:11.4f}'.format('DGintAB-A-B', total))
print('\nWritting done in file: inter_en_res.csv\n')
print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------

TOTAL ENERGIES of interaction energy based in interface residues ONLY
Total Elec Int.     :     19.9753
Total Vdw Int.      :   -143.9898
Total Solv AB       :    -10.5493
Total Solv         A:    -29.1297
Total Solv         E:     13.8159
DGintAB-A-B         :   -119.2500

Writting done in file: inter_en_res.csv

-----------------------------------------------------------------------


Alanine scaning for DDGs X->Ala mutations on the interface residues

In [23]:
print('-------------------------------------------------------------------------')
print(f'\nAla Scanning: DDGs for X->Ala mutations on interface residues')

# # Opening/creating the file in writting mode
with open("ala_scaning.csv", "w") as file:

	# Header with format (as output in the terminal & write in tha file):
	file.write(
		'{:11},{:11s},{:11s},{:11s},{:11s},{:11s}\n'.format(
		'res_id',
		'elec',
		'vdw',
		'solvAB',
		'solv',
		'total'))

	print(
		'{:11}  {:11s} {:11s} {:11s} {:11s} {:11s}'.format(
			'res_id',
			'elec',
			'vdw',
			'solvAB',
			'solv',
			'total'))

	# Iteration through the chain
	for ch in st[0]:

		# Iteration through all the residuals
		for res in ch.get_residues():

			# Making sure the distance is not 0 (deesn't make sense)
			if cutoff_dist > 0 and res not in interface[ch.id]:
				continue

			print(
				'{:11}  {:11.4f} {:11.4f} {:11.4f} {:11.4f} {:11.4f}'.format(
					en.residue_id(res),
					- elec[res] + elec_ala[res],
					- vdw[res] + vdw_ala[res],
					- solvAB[res] + solvAB_ala[res],
					- solvA[res] + solvA_ala[res],
					- elec[res] + elec_ala[res] - vdw[res] + vdw_ala[res] -solvAB[res] +\
					solvAB_ala[res] -solvA[res] + solvA_ala[res]))
			
			# Main body of all the info for each residue with it's csv format (as output in the terminal & write in tha file)
			file.write(
				'{:11},{:11.4f},{:11.4f},{:11.4f},{:11.4f},{:11.4f}\n'.format(
					en.residue_id(res),
					- elec[res] + elec_ala[res],
		   		 - vdw[res] + vdw_ala[res],
		  		  - solvAB[res] + solvAB_ala[res],
		  		  - solvA[res] + solvA_ala[res],
		  		  - elec[res] + elec_ala[res] - vdw[res] + vdw_ala[res] -solvAB[res] +\
		  			  solvAB_ala[res] -solvA[res] + solvA_ala[res]))

print('Writing  done in file: ala_scaning.csv')
print('\n-------------------------------------------------------------------------')

-------------------------------------------------------------------------

Ala Scanning: DDGs for X->Ala mutations on interface residues
res_id       elec        vdw         solvAB      solv        total      
SER A19          -1.2151      0.0838      1.2701      1.2701      1.4090
GLN A24          -0.5145     -0.5396      1.8081      2.5306      3.2846
THR A27           1.2889      3.1875      0.7692     -0.2673      4.9784
PHE A28           0.0887      1.3642     -0.5991     -1.6069     -0.7530
ASP A30           1.9425      2.5027      1.7551      3.4265      9.6268
LYS A31          -4.7766      5.0141      2.2068      1.7994      4.2437
HIE A34           0.1917      0.5240      1.6992      3.0397      5.4545
GLU A35           3.4826      0.1963      1.8335      2.7878      8.3002
GLU A37           4.1348      2.0987      0.9054      1.5457      8.6846
ASP A38           2.4785      1.4971      0.8910      2.5722      7.4389
TYR A41           0.3144      5.2696      0.0000     -1.5778